Combinacion de modelos 

In [1]:
# Importar librerías necesarias
import pandas as pd
import numpy as np

print("Librerías importadas correctamente")
print("Combinación: Primeros 300 productos de AutoGluon+RL, resto de Media 12 093")

Librerías importadas correctamente
Combinación: Primeros 300 productos de AutoGluon+RL, resto de Media 12 093


In [2]:
# Cargar los archivos de predicciones
print("Cargando archivos de predicciones...")

# Cargar predicciones del ensemble AutoGluon + Regresión Lineal
pred_autogluon_rl = pd.read_csv('pred_autogluon_rl_01.csv')
print(f"AutoGluon+RL Ensemble - Shape: {pred_autogluon_rl.shape}")
print("AutoGluon+RL Ensemble - Primeras filas:")
print(pred_autogluon_rl.head())

print("\n" + "="*50 + "\n")

# Cargar predicciones de Media 12 093
pred_media = pd.read_csv('data/pred_medias_12_093.csv')
print(f"Media 12 093 - Shape: {pred_media.shape}")
print("Media 12 093 - Primeras filas:")
print(pred_media.head())

# Verificar que ambos archivos tienen product_ids ordenados
print(f"\nProduct_id mínimo AutoGluon+RL: {pred_autogluon_rl['product_id'].min()}")
print(f"Product_id máximo AutoGluon+RL: {pred_autogluon_rl['product_id'].max()}")
print(f"Product_id mínimo Media 12 093: {pred_media['product_id'].min()}")
print(f"Product_id máximo Media 12 093: {pred_media['product_id'].max()}")

Cargando archivos de predicciones...
AutoGluon+RL Ensemble - Shape: (780, 2)
AutoGluon+RL Ensemble - Primeras filas:
   product_id           tn
0       20001  1242.745126
1       20002  1129.899731
2       20003   690.468400
3       20004   549.851377
4       20005   532.285525


Media 12 093 - Shape: (780, 2)
Media 12 093 - Primeras filas:
   product_id           tn
0       20001  1352.901430
1       20002  1093.156542
2       20003   730.028059
3       20004   583.310255
4       20005   621.491197

Product_id mínimo AutoGluon+RL: 20001
Product_id máximo AutoGluon+RL: 21276
Product_id mínimo Media 12 093: 20001
Product_id máximo Media 12 093: 21276


In [3]:
# Ordenar los datos por product_id para garantizar consistencia
print("Ordenando datos por product_id...")

# Ordenar ambos dataframes por product_id
pred_autogluon_rl = pred_autogluon_rl.sort_values('product_id').reset_index(drop=True)
pred_media = pred_media.sort_values('product_id').reset_index(drop=True)

print("Datos ordenados correctamente")

# Verificar que ambos archivos tienen los mismos product_ids
productos_autogluon_rl = set(pred_autogluon_rl['product_id'])
productos_media = set(pred_media['product_id'])

if productos_autogluon_rl == productos_media:
    print("✅ Ambos archivos tienen exactamente los mismos productos")
    productos_totales = len(productos_autogluon_rl)
    print(f"Total de productos disponibles: {productos_totales}")
else:
    print("⚠️ Los archivos NO tienen exactamente los mismos productos")
    productos_comunes = productos_autogluon_rl & productos_media
    print(f"Productos comunes: {len(productos_comunes)}")
    
print(f"\nPrimeros 5 product_ids ordenados:")
print(pred_autogluon_rl['product_id'].head().tolist())

Ordenando datos por product_id...
Datos ordenados correctamente
✅ Ambos archivos tienen exactamente los mismos productos
Total de productos disponibles: 780

Primeros 5 product_ids ordenados:
[20001, 20002, 20003, 20004, 20005]


In [4]:
# Crear la combinación: primeros 300 productos del ensemble, resto de regresión lineal
print("Creando combinación de modelos...")

# Definir el punto de corte
corte_productos = 300
total_productos = len(pred_autogluon_rl)

print(f"Estrategia de combinación:")
print(f"  - Primeros {corte_productos} productos: AutoGluon+RL Ensemble")
print(f"  - Productos {corte_productos+1} a {total_productos}: Media 12 093")

# Tomar los primeros 300 productos del ensemble
primeros_300 = pred_autogluon_rl.head(corte_productos).copy()
primeros_300['origen'] = 'AutoGluon+RL_Ensemble'

# Tomar los productos restantes de media 12 093
productos_restantes = pred_media.iloc[corte_productos:].copy()
productos_restantes['origen'] = 'Media_12_093'

print(f"\nPrimeros 300 productos (del ensemble):")
print(f"  Shape: {primeros_300.shape}")
print(f"  Product_id rango: {primeros_300['product_id'].min()} - {primeros_300['product_id'].max()}")

print(f"\nProductos restantes (de media 12 093):")
print(f"  Shape: {productos_restantes.shape}")
print(f"  Product_id rango: {productos_restantes['product_id'].min()} - {productos_restantes['product_id'].max()}")

# Combinar ambos conjuntos
combinacion_final = pd.concat([primeros_300, productos_restantes], ignore_index=True)

print(f"\nCombinación final:")
print(f"  Shape total: {combinacion_final.shape}")
print(f"  Productos del ensemble: {(combinacion_final['origen'] == 'AutoGluon+RL_Ensemble').sum()}")
print(f"  Productos de media 12 093: {(combinacion_final['origen'] == 'Media_12_093').sum()}")

Creando combinación de modelos...
Estrategia de combinación:
  - Primeros 300 productos: AutoGluon+RL Ensemble
  - Productos 301 a 780: Media 12 093

Primeros 300 productos (del ensemble):
  Shape: (300, 3)
  Product_id rango: 20001 - 20365

Productos restantes (de media 12 093):
  Shape: (480, 3)
  Product_id rango: 20366 - 21276

Combinación final:
  Shape total: (780, 3)
  Productos del ensemble: 300
  Productos de media 12 093: 480


In [5]:
# Mostrar ejemplos de la transición entre modelos
print("ANÁLISIS DE LA TRANSICIÓN ENTRE MODELOS")
print("="*50)

# Mostrar los últimos productos del ensemble y los primeros de regresión
print("Últimos 5 productos del ensemble (AutoGluon+RL):")
transicion_ensemble = combinacion_final[combinacion_final['origen'] == 'AutoGluon+RL_Ensemble'].tail()
print(transicion_ensemble[['product_id', 'tn', 'origen']])

print(f"\nPrimeros 5 productos de media 12 093:")
transicion_media = combinacion_final[combinacion_final['origen'] == 'Media_12_093'].head()
print(transicion_media[['product_id', 'tn', 'origen']])

# Comparar estadísticas entre ambos grupos
print(f"\nESTADÍSTICAS COMPARATIVAS:")
stats_ensemble = combinacion_final[combinacion_final['origen'] == 'AutoGluon+RL_Ensemble']['tn']
stats_media = combinacion_final[combinacion_final['origen'] == 'Media_12_093']['tn']

print(f"\nPrimeros 300 productos (AutoGluon+RL Ensemble):")
print(f"  Promedio: {stats_ensemble.mean():.2f}")
print(f"  Mediana:  {stats_ensemble.median():.2f}")
print(f"  Std:      {stats_ensemble.std():.2f}")
print(f"  Min:      {stats_ensemble.min():.2f}")
print(f"  Max:      {stats_ensemble.max():.2f}")

print(f"\nProductos restantes (Media 12 093):")
print(f"  Promedio: {stats_media.mean():.2f}")
print(f"  Mediana:  {stats_media.median():.2f}")
print(f"  Std:      {stats_media.std():.2f}")
print(f"  Min:      {stats_media.min():.2f}")
print(f"  Max:      {stats_media.max():.2f}")

print(f"\nCombinación completa:")
print(f"  Promedio: {combinacion_final['tn'].mean():.2f}")
print(f"  Mediana:  {combinacion_final['tn'].median():.2f}")
print(f"  Std:      {combinacion_final['tn'].std():.2f}")

ANÁLISIS DE LA TRANSICIÓN ENTRE MODELOS
Últimos 5 productos del ensemble (AutoGluon+RL):
     product_id         tn                 origen
295       20359  13.314429  AutoGluon+RL_Ensemble
296       20361  16.816129  AutoGluon+RL_Ensemble
297       20362   8.037368  AutoGluon+RL_Ensemble
298       20364  25.532795  AutoGluon+RL_Ensemble
299       20365  13.136061  AutoGluon+RL_Ensemble

Primeros 5 productos de media 12 093:
     product_id         tn        origen
300       20366  11.859593  Media_12_093
301       20367  11.330856  Media_12_093
302       20368  17.731964  Media_12_093
303       20372   8.723977  Media_12_093
304       20375  10.251801  Media_12_093

ESTADÍSTICAS COMPARATIVAS:

Primeros 300 productos (AutoGluon+RL Ensemble):
  Promedio: 85.46
  Mediana:  40.51
  Std:      135.66
  Min:      8.04
  Max:      1242.75

Productos restantes (Media 12 093):
  Promedio: 5.00
  Mediana:  3.19
  Std:      5.13
  Min:      0.02
  Max:      29.08

Combinación completa:
  Promedio:

In [6]:
# Preparar y guardar el archivo final
print("Preparando archivo final...")

# Crear el resultado final con solo product_id y tn (sin la columna 'origen')
resultado_final = combinacion_final[['product_id', 'tn']].copy()

# Verificar que está ordenado por product_id
resultado_final = resultado_final.sort_values('product_id').reset_index(drop=True)

# Verificar que no hay valores nulos
print(f"Valores nulos en el resultado final: {resultado_final.isnull().sum().sum()}")
print(f"Shape del resultado final: {resultado_final.shape}")

print(f"\nPrimeras 10 filas del resultado final:")
print(resultado_final.head(10))

print(f"\nÚltimas 5 filas del resultado final:")
print(resultado_final.tail())

# Verificar continuidad en los product_ids
print(f"\nVerificación de continuidad:")
print(f"Product_id mínimo: {resultado_final['product_id'].min()}")
print(f"Product_id máximo: {resultado_final['product_id'].max()}")
print(f"Total de productos: {len(resultado_final)}")

# Punto de transición (producto 300)
producto_300 = resultado_final.iloc[299]  # índice 299 = producto 300
producto_301 = resultado_final.iloc[300]  # índice 300 = producto 301
print(f"\nPunto de transición:")
print(f"Producto {corte_productos} (último del ensemble): ID {producto_300['product_id']}, tn = {producto_300['tn']:.2f}")
print(f"Producto {corte_productos+1} (primero de media): ID {producto_301['product_id']}, tn = {producto_301['tn']:.2f}")

Preparando archivo final...
Valores nulos en el resultado final: 0
Shape del resultado final: (780, 2)

Primeras 10 filas del resultado final:
   product_id           tn
0       20001  1242.745126
1       20002  1129.899731
2       20003   690.468400
3       20004   549.851377
4       20005   532.285525
5       20006   467.134765
6       20007   394.212203
7       20008   406.082441
8       20009   470.562106
9       20010   400.904045

Últimas 5 filas del resultado final:
     product_id        tn
775       21263  0.027894
776       21265  0.083273
777       21266  0.088033
778       21267  0.086337
779       21276  0.042266

Verificación de continuidad:
Product_id mínimo: 20001
Product_id máximo: 21276
Total de productos: 780

Punto de transición:
Producto 300 (último del ensemble): ID 20365.0, tn = 13.14
Producto 301 (primero de media): ID 20366.0, tn = 11.86


In [7]:
# Guardar el archivo final
archivo_salida = "data/pred_combina_autogluon300_media_12_093.csv"

print(f"Guardando combinación de modelos en: {archivo_salida}")

# Crear directorio data si no existe
import os
os.makedirs("data", exist_ok=True)

# Guardar el archivo
resultado_final.to_csv(archivo_salida, index=False)

print(f"✅ Archivo guardado exitosamente: {archivo_salida}")
print(f"Total de productos: {len(resultado_final)}")

# Verificar que el archivo se guardó correctamente
if os.path.exists(archivo_salida):
    tamaño_archivo = os.path.getsize(archivo_salida)
    print(f"Tamaño del archivo: {tamaño_archivo} bytes")
    
    # Leer las primeras y últimas líneas del archivo guardado para verificar
    verificacion_inicio = pd.read_csv(archivo_salida, nrows=5)
    print(f"\nVerificación - Primeras 5 líneas del archivo guardado:")
    print(verificacion_inicio)
    
    # Verificar los productos cerca del punto de transición
    verificacion_completa = pd.read_csv(archivo_salida)
    print(f"\nVerificación del punto de transición (alrededor del producto 300):")
    transicion_verificacion = verificacion_completa.iloc[297:303]  # productos 298-303
    print(transicion_verificacion)
    
else:
    print("⚠️ Error: No se pudo crear el archivo")

Guardando combinación de modelos en: data/pred_combina_autogluon300_media_12_093.csv
✅ Archivo guardado exitosamente: data/pred_combina_autogluon300_media_12_093.csv
Total de productos: 780
Tamaño del archivo: 17804 bytes

Verificación - Primeras 5 líneas del archivo guardado:
   product_id           tn
0       20001  1242.745126
1       20002  1129.899731
2       20003   690.468400
3       20004   549.851377
4       20005   532.285525

Verificación del punto de transición (alrededor del producto 300):
     product_id         tn
297       20362   8.037368
298       20364  25.532795
299       20365  13.136061
300       20366  11.859593
301       20367  11.330856
302       20368  17.731964


In [8]:
# Resumen final de la combinación
print("RESUMEN FINAL DE LA COMBINACIÓN DE MODELOS")
print("="*60)
print(f"📁 Archivo fuente 1:        pred_autogluon_rl_01.csv (primeros 300)")
print(f"📁 Archivo fuente 2:        data/pred_medias_12_093.csv (restantes)")
print(f"📁 Archivo generado:        {archivo_salida}")
print(f"📊 Total de productos:      {len(resultado_final)}")

print(f"\n🔄 ESTRATEGIA DE COMBINACIÓN:")
print(f"   • Productos 1-{corte_productos}:    AutoGluon+RL Ensemble")
print(f"   • Productos {corte_productos+1}-{len(resultado_final)}:   Media 12 093")

print(f"\n📈 ESTADÍSTICAS POR SEGMENTO:")
print(f"   Ensemble (1-{corte_productos}):")
print(f"     - Promedio: {stats_ensemble.mean():.2f}")
print(f"     - Rango: {stats_ensemble.min():.2f} - {stats_ensemble.max():.2f}")

print(f"   Media 12 093 ({corte_productos+1}-{len(resultado_final)}):")
print(f"     - Promedio: {stats_media.mean():.2f}")
print(f"     - Rango: {stats_media.min():.2f} - {stats_media.max():.2f}")

print(f"\n🎯 JUSTIFICACIÓN:")
print(f"   Esta combinación permite usar las predicciones del ensemble")
print(f"   (más robustas) para los primeros productos, mientras que")
print(f"   utiliza medias históricas para el resto, creando un")
print(f"   enfoque híbrido que balancea precisión y estabilidad.")

print(f"\n✅ Combinación completada exitosamente!")
print(f"   El archivo {archivo_salida} está listo para usar.")

RESUMEN FINAL DE LA COMBINACIÓN DE MODELOS
📁 Archivo fuente 1:        pred_autogluon_rl_01.csv (primeros 300)
📁 Archivo fuente 2:        data/pred_medias_12_093.csv (restantes)
📁 Archivo generado:        data/pred_combina_autogluon300_media_12_093.csv
📊 Total de productos:      780

🔄 ESTRATEGIA DE COMBINACIÓN:
   • Productos 1-300:    AutoGluon+RL Ensemble
   • Productos 301-780:   Media 12 093

📈 ESTADÍSTICAS POR SEGMENTO:
   Ensemble (1-300):
     - Promedio: 85.46
     - Rango: 8.04 - 1242.75
   Media 12 093 (301-780):
     - Promedio: 5.00
     - Rango: 0.02 - 29.08

🎯 JUSTIFICACIÓN:
   Esta combinación permite usar las predicciones del ensemble
   (más robustas) para los primeros productos, mientras que
   utiliza medias históricas para el resto, creando un
   enfoque híbrido que balancea precisión y estabilidad.

✅ Combinación completada exitosamente!
   El archivo data/pred_combina_autogluon300_media_12_093.csv está listo para usar.
